#08 - Error Analysis

To evaluate the performance of both activation functions with respect to one another, I decided to generate contingency matrices. I modeled the identification of false postives and false negatives by Sigmoid and ReLU. I found that ReLU's robustness led it to predict falsely more often than Sigmoid; however, similarities among both functions' false predictions suggest that the dataset contained some training examples intrinsically difficult to learn.

In [ ]:
#Importing all existing network configurations
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent / "src"))
from utils import make_mlp, set_seed, plot_loss

In [ ]:
def collectExamples(model, dataloader, threshold):
  #Running model in evaluation mode
  model.eval()
  with torch.no_grad():
    val_labels = []
    val_predictions = []
    val_outputs = []
    #Calculating accuracy on validation data set
    for val_x, val_y in dataloader:

      outputs = model(val_x)
      predictions = (outputs >= threshold).int()
      val_labels.append(val_y)
      val_outputs.append(outputs)
      val_predictions.append(predictions)

    #Recording model prediction vs true outputs
    val_labels = torch.cat(val_labels, dim = 0)
    val_predictions = torch.cat(val_predictions, dim = 0)
    val_outputs = torch.cat(val_outputs, dim = 0)
    return val_labels, val_predictions, val_outputs

val_labels, val_predictions, val_outputs = collectExamples(model, val_dataloader, val_best_threshold)

In [ ]:
#Applying logic masks to find false positives and false negatives of both activation functions
relu_fp_mask = (val_labels == 0) & (val_predictions == 1)
relu_fn_mask = (val_labels == 1) & (val_predictions == 0)

sig_fp_mask = (val_labels == 0) & (val_predictions == 1)
sig_fn_mask = (val_labels == 1) & (val_predictions == 0)

In [ ]:
#Printing data for tabulation
print("False positives: \n")
print(sig_fp_mask.sum(), relu_fp_mask.sum())

print((sig_fp_mask & relu_fp_mask).sum())
print(((~relu_fp_mask) & sig_fp_mask).sum())
print((relu_fp_mask & (~sig_fp_mask)).sum())
print(((~relu_fp_mask) & (~sig_fp_mask)).sum())


print("\n False negatives: \n")
print(sig_fn_mask.sum(), relu_fn_mask.sum())

print((sig_fn_mask & relu_fn_mask).sum())
print(((~relu_fn_mask) & sig_fn_mask).sum())
print((relu_fn_mask & (~sig_fn_mask)).sum())
print(((~relu_fn_mask) & (~sig_fn_mask)).sum())


False positives: 

tensor(189) tensor(218)
tensor(157)
tensor(32)
tensor(61)
tensor(790)

 False negatives: 

tensor(60) tensor(44)
tensor(30)
tensor(30)
tensor(14)
tensor(966)
